In [1]:
import pandas as pd
import numpy as np
import math
import string
import copy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from collections import Counter
from tqdm import tqdm
from tensorflow_addons.metrics import F1Score
import warnings
import os
warnings.filterwarnings("ignore")

In [41]:
stfold = StratifiedKFold(n_splits=5, shuffle=True)
train_l = list()
val_l = list()
test_l = list()
maxlen = 100
cnt = 0
for train_index, test_index in stfold.split(data['comment'], data['label']):
    cnt += 1
    train = data.iloc[train_index]
    train_x, train_y = train['comment'], train['label']
    tmp = data.iloc[test_index]

    val, test = train_test_split(tmp, test_size=0.5, stratify=tmp['label'])    
    val_x, val_y = val['comment'], val['label']
    test_x, test_y = test['comment'], test['label']    
    
    train_x = keras.preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
    val_x = keras.preprocessing.sequence.pad_sequences(val_x, maxlen=maxlen)
    test_x = keras.preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)
    
    embed_dim = 32  # Embedding size for each token
    vocab_size = n_word_unique + 1

    inputs = layers.Input(shape=(maxlen, ))
    embedding_layer = layers.Embedding(input_dim = vocab_size, output_dim = embed_dim, input_length=maxlen)
    x = embedding_layer(inputs)
    x = tf.keras.layers.Flatten()(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model_path = "sigmoid_models/{itr:04}/".format(itr=cnt)
    checkpoint_path = model_path + "{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    !mkdir sigmoid_models
    !mkdir $model_path
    
    
    model.compile('adam', "binary_crossentropy", metrics=["accuracy", 
                                                          tf.keras.metrics.AUC(name='auc')])
    
    
    #create checkpoint to save model
    #with best validation loss
    model.save_weights(checkpoint_path.format(epoch=0))

    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1,
            save_weights_only=True, save_best_only=True, mode='auto')
    
    history = model.fit(
                        train_x, train_y, 
                        batch_size=128, epochs=10, 
                        validation_data=(val_x, val_y),
                        callbacks = [checkpoint]
                        )

    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)
    
    train_eval = model.evaluate(train_x, train_y)
    val_eval = model.evaluate(val_x, val_y)
    test_eval = model.evaluate(test_x, test_y)
    
    test_l.append(test_eval)
    val_l.append(val_eval)
    train_l.append(train_eval)

Epoch 1/10
62/63 [============================>.] - ETA: 0s - loss: 0.6860 - accuracy: 0.5431 - auc: 0.5571
Epoch 00001: val_loss improved from inf to 0.67839, saving model to sigmoid_models/0001/0001.ckpt
63/63 [==============================] - 1s 15ms/step - loss: 0.6858 - accuracy: 0.5439 - auc: 0.5580 - val_loss: 0.6784 - val_accuracy: 0.5790 - val_auc: 0.6313
Epoch 2/10
60/63 [===========================>..] - ETA: 0s - loss: 0.6446 - accuracy: 0.6482 - auc: 0.7080
Epoch 00002: val_loss improved from 0.67839 to 0.65377, saving model to sigmoid_models/0001/0002.ckpt
63/63 [==============================] - 1s 9ms/step - loss: 0.6439 - accuracy: 0.6494 - auc: 0.7085 - val_loss: 0.6538 - val_accuracy: 0.6150 - val_auc: 0.6770
Epoch 3/10
59/63 [===========================>..] - ETA: 0s - loss: 0.5399 - accuracy: 0.8245 - auc: 0.8973
Epoch 00003: val_loss improved from 0.65377 to 0.60675, saving model to sigmoid_models/0001/0003.ckpt
63/63 [==============================] - 1s 9ms/ste

In [42]:
test_l = np.array(test_l)
val_l = np.array(val_l)
train_l = np.array(train_l)

print("test avg loss: ", np.mean(test_l[:, 0]), "+/-" ,np.std(test_l[:, 0]))
print("test avg acc: ", np.mean(test_l[:, 1]),  "+/-" ,np.std(test_l[:, 1]))
print("test avg auc: ", np.mean(test_l[:, 2]),  "+/-" ,np.std(test_l[:, 2]))

print('\n')


print("val avg loss: ", np.mean(val_l[:, 0]), "+/-" ,np.std(val_l[:, 0]))
print("val avg acc: ", np.mean(val_l[:, 1]),  "+/-" ,np.std(val_l[:, 1]))
print("val avg auc: ", np.mean(val_l[:, 2]),  "+/-" ,np.std(val_l[:, 2]))


print('\n')


print("train avg loss: ", np.mean(train_l[:, 0]), "+/-" ,np.std(train_l[:, 0]))
print("train avg acc: ", np.mean(train_l[:, 1]),  "+/-" ,np.std(train_l[:, 1]))
print("train avg auc: ", np.mean(train_l[:, 2]),  "+/-" ,np.std(train_l[:, 2]))

test avg loss:  0.45945392847061156 +/- 0.022896583071277805
test avg acc:  0.7814000010490417 +/- 0.016475431373525955
test avg auc:  0.8627335786819458 +/- 0.01668934624429813


val avg loss:  0.47193355560302735 +/- 0.013726163877872014
val avg acc:  0.7742000102996827 +/- 0.01390540215606582
val avg auc:  0.8525553941726685 +/- 0.0091138210505089


train avg loss:  0.032654179632663725 +/- 0.0035417074225733954
train avg acc:  0.9991999983787536 +/- 0.00012746624851231656
train avg auc:  0.999998128414154 +/- 9.52630664062836e-07
